<a href="https://colab.research.google.com/github/Nikkutan/Nikhil/blob/main/Fake_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

**Data Analysis**

In [ ]:
import csv
csv.field_size_limit(100000000)

131072

In [ ]:
df=pd.read_csv('WELFake_Dataset.csv',error_bad_lines=False, engine="python")

<ipython-input-109-660e21388c23>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('WELFake_Dataset.csv',error_bad_lines=False, engine="python")
Skipping line 28705: unexpected end of data


In [ ]:
df

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...,...,...
28698,28698,Honduras opposition candidate calls on U.S. no...,WASHINGTON (Reuters) - Honduran opposition can...,0
28699,28699,Rights group voices fears over Trump refugee p...,GENEVA (Reuters) - U.S. President Donald Trump...,0
28700,28700,World War 3?,,1
28701,28701,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,0


In [ ]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28703 entries, 0 to 28702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  28703 non-null  int64 
 1   title       28520 non-null  object
 2   text        28689 non-null  object
 3   label       28703 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 897.1+ KB


There are null values and want to know how many null values are therer

In [ ]:
df.isnull().sum()

Unnamed: 0      0
title         183
text           14
label           0
dtype: int64

As the Dtype of the title and text columns are object, we need to drop those null values

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(28506, 4)

As this is a classification problem and the label is the column of our interset, we need to check whether the data is balanced or not . Can use value_counts()

In [ ]:
df['label'].value_counts()

1    14678
0    13828
Name: label, dtype: int64

The data is a balanced data.

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head()

,index,Unnamed: 0,title,text,label
0,0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
2,3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
3,4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
4,5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [ ]:
df['text'][0]

'No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to  turn the tide  and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called  Sunshine.  She has a radio blog show hosted from Texas called,  Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to  Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite   #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show, callers clearly call for  lynching  and  killing  of white people.A 2:39 minute clip from the radio show can be heard here. It was provided to Breitbart Texas by someone who would like to be referred to

In [ ]:
df['title'][0]

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [ ]:
df.drop(['Unnamed: 0','text','index'],axis=1,inplace=True)

In [ ]:
df.head()

,title,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0
3,SATAN 2: Russia unvelis an image of its terrif...,1
4,About Time! Christian Group Sues Amazon and SP...,1


**Toeknization Stopword-Removal Lemmatization**

In [ ]:
df['title'][0]

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))

The sentance needs to be tokenized and there is lower and upper case, want to convert it to lower case, remove special characters with regular expression,and then remove the stopwords present in sentance.

In [ ]:
tokens=[]

In [ ]:
for i in range (len(df)):
    review = re.sub('^a-zA-Z0-9',' ', df['title'][i])
    review = review.lower()
    review = review.split()
    tokens.append(review)

In [ ]:
lm = WordNetLemmatizer()
corpus = []

for j in tokens:
    res = [lm.lemmatize(i) for i in j if i not in stop_words]
    res = ' '.join(res)
    corpus.append(res)

In [ ]:
df['title'][0]

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [ ]:
corpus[0]

'law enforcement high alert following threat cop white 9-11by #blacklivesmatter #fyf911 terrorist [video]'

**TF-IDF Vectorization**

In [ ]:
tf = TfidfVectorizer()
x=tf.fit_transform(corpus).toarray()

In [ ]:
y=df['label']
y.head()

0    1
1    1
2    0
3    1
4    1
Name: label, dtype: int64

**Train Test Split**

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=10,stratify=y)

Here we are using the stratify=y as we need equal number of 1 and 0.

**Model Building**

Here Random Forest classifier is used to build model

In [ ]:
rf=RandomForestClassifier()

In [ ]:
rf.fit(x_train,y_train)

RandomForestClassifier()

**Model Evaluation**

In [ ]:
y_pred=rf.predict(x_test)

In [ ]:
accuracy=accuracy_score(y_test,y_pred)

In [ ]:
accuracy

0.8758185219831618

We get an accuracy of 0.876 which is very good classification model and now create a Evaluation report of both training and testing data.

In [ ]:
class Evaluation:
  def __init__(self,model,x_train,x_test,y_train,y_test):
    self.model=model
    self.x_train=x_train
    self.x_test=x_test
    self.y_train=y_train
    self.y_test=y_test

  def Training_Data_Evaluation(self):

    y_pred_train=self.model.predict(self.x_train)
    Training_data_Accuracy=accuracy_score(self.y_train,y_pred_train)
    print('Training data accuracy:',Training_data_Accuracy)
    print()

    Training_data_confusion_matrix=confusion_matrix(self.y_train,y_pred_train)
    print('Training data confusion matrix:\n',Training_data_confusion_matrix)
    print()

    Training_data_classification_report=classification_report(self.y_train,y_pred_train)
    print('Training data classification report:\n',Training_data_classification_report)

  def Testing_Data_Evaluation(self):

    y_pred_test=self.model.predict(self.x_test)
    Testing_data_Accuracy=accuracy_score(self.y_test,y_pred_test)
    print('Testing data accuracy:',Testing_data_Accuracy)
    print()

    Testing_data_confusion_matrix=confusion_matrix(self.y_test,y_pred_test)
    print('Testing data confusion matrix:\n',Testing_data_confusion_matrix)
    print()

    Testing_data_classification_report=classification_report(self.y_test,y_pred_test)
    print('Testing data classification report:\n',Testing_data_classification_report)

In [ ]:
Evaluation(rf,x_train, x_test, y_train, y_test).Training_Data_Evaluation()

Training data accuracy: 0.9998997694697805

Training data confusion matrix:
 [[ 9678     2]
 [    0 10274]]

Training data classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9680
           1       1.00      1.00      1.00     10274

    accuracy                           1.00     19954
   macro avg       1.00      1.00      1.00     19954
weighted avg       1.00      1.00      1.00     19954



In [ ]:
Evaluation(rf,x_train, x_test, y_train, y_test).Testing_Data_Evaluation()

Testing data accuracy: 0.8758185219831618

Testing data confusion matrix:
 [[3539  609]
 [ 453 3951]]

Testing data classification report:
               precision    recall  f1-score   support

           0       0.89      0.85      0.87      4148
           1       0.87      0.90      0.88      4404

    accuracy                           0.88      8552
   macro avg       0.88      0.88      0.88      8552
weighted avg       0.88      0.88      0.88      8552



**News Prediction**

In [ ]:
class Preprocessing:

    def __init__(self,data):
        self.data = data

    def text_preprocessing_user(self):
        lm = WordNetLemmatizer()
        pred_data = [self.data]
        preprocessed_data = []
        for data in pred_data:
            reviews = re.sub('^a-zA-Z0-9',' ', data)
            reviews = reviews.lower()
            reviews = reviews.split()
            reviews = [lm.lemmatize(x) for x in reviews if x not in stop_words]
            reviews = " ".join(reviews)
            preprocessed_data.append(reviews)
        return preprocessed_data

class Prediction:

    def __init__(self,pred_data, model):
        self.pred_data = pred_data
        self.model = model

    def prediction_model(self):
        preprocessed_data = Preprocessing(self.pred_data).text_preprocessing_user()
        data = tf.transform(preprocessed_data)
        prediction = self.model.predict(data)

        if prediction [0] == 0 :
            return "The News Is Fake"

        else:
            return "The News Is Real"

In [ ]:
df['title'][0]

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [ ]:
data='LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [ ]:
Preprocessing(data).text_preprocessing_user()

['law enforcement high alert following threat cop white 9-11by #blacklivesmatter #fyf911 terrorist [video]']

In [ ]:
Prediction(data,rf).prediction_model()

'The News Is Real'